## Setup

In [ ]:
!pip uninstall pandas-profiling

In [ ]:
#!pip install pandas-profiling
#!pip install --upgrade pandas-profiling


In [ ]:
import nltk
import spacy
import pandas as pd
import numpy as np
import pandas_profiling as prof

from pathlib import Path
from textblob import TextBlob

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 6.0)
import plotly.graph_objs as go

import cufflinks as cf
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot
from IPython.core.interactiveshell import InteractiveShell

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

InteractiveShell.ast_node_interactivity = 'all'
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True, theme='pearl')

pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 30


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Exploratory Data Analysis (EDA)


You can do EDA using this simple method: Pandas Profiling
Generates profile reports from a pandas DataFrame. The pandas df.describe() function is great but a little basic for serious exploratory data analysis. pandas_profiling extends the pandas DataFrame with df.profile_report() for quick data analysis.

For each column the following statistics - if relevant for the column type - are presented in an interactive HTML report:

* Type inference: detect the types of columns in a dataframe.
* Essentials: type, unique values, missing values
* Quantile statistics like minimum value, Q1, median, Q3, maximum, range, interquartile range
* Descriptive statistics like mean, mode, standard deviation, sum, median absolute deviation, coefficient of variation, kurtosis, skewness
* Most frequent values
* Histogram
* Correlations highlighting of highly correlated variables, Spearman, Pearson and Kendall matrices
* Missing values matrix, count, heatmap and dendrogram of missing values
* Text analysis: learn about categories (Uppercase, Space), scripts (Latin, Cyrillic) and blocks (ASCII) of text data.
---


Play & Learn

> Learn more about data profiling: https://github.com/pandas-profiling/pandas-profiling
> Dive into sentiment analysis with TextBlob: https://textblob.readthedocs.io/en/dev/

In [ ]:
# Load dataset
data_folder = Path("/content/drive/MyDrive/CLT/S2/data/")
clothing_reviews = data_folder / "Womens Clothing E-Commerce Reviews.csv"

### Manual EDA
with Pandas built-in methods

Overview:
* DataFrame.count: Count number of non-NA/null observations.
* DataFrame.max: Maximum of the values in the object.
* DataFrame.min: Minimum of the values in the object.
* DataFrame.mean: Mean of the values.
* DataFrame.std: Standard deviation of the observations.
* DataFrame.select_dtypes: Subset of a DataFrame including/excluding           columns based on their dtype.

In [ ]:
# Inspect the data frame
df = pd.read_csv(clothing_reviews)
df.head(10)

In [ ]:
# Inspect the dimensionality of your DataFrame. The shape attribute of pandas.DataFrame stores the number of rows and columns as a tuple (number of rows, number of columns).
df.shape

In [ ]:
# Remove unused coloumns
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# Print information about your DataFrame. You can inspect the total memory usage, the data type of each column, and the number of non-NaN elements.
df.info()

In [ ]:
# Remove 'Title' feature <- many titles are missing
df.drop('Title', axis=1, inplace=True)

In [ ]:
# Remove 'Review Text' without text
df.dropna(subset=['Review Text'], inplace=True)

# Remove rows with missing 'Department Name'
df.dropna(subset=['Department Name'], inplace=True)

In [ ]:
# Clean up the 'Review Text' data a little
def preprocess(text):
    text = text.str.replace("(<br/>)", "")
    text = text.str.replace('(<a).*(>).*(</a>)', '')
    text = text.str.replace('(&amp)', '')
    text = text.str.replace('(&gt)', '')
    text = text.str.replace('(&lt)', '')
    text = text.str.replace('(\xa0)', ' ')  
    return text

df['Review Text'] = preprocess(df['Review Text'])

In [ ]:
# We will add some basic text features to the data

# Add 'Review lenght'
df['Review Length'] = df['Review Text'].astype(str).apply(len)

# Add simple token count
df['Num Tokens'] = df['Review Text'].apply(lambda x: len(str(x).split()))

# Add polarity
df['Polarity'] = df['Review Text'].map(lambda text: TextBlob(str(text)).sentiment.polarity)

We are using a simple sentiment library, `TextBlob` to calculate sentiment polarity which lies in the range of [-1,1] where **1 means positive** sentiment and **-1 means a negative** sentiment.

In [ ]:
# Sentiment examples
print (TextBlob('I love chocolate.').sentiment.polarity)
print (TextBlob('I hate chocolate.').sentiment.polarity)    

In [ ]:
df.info()

In [ ]:
# Descriptive statistics of your dataframe
df.describe()

In [ ]:
# Take a look at our new features
df.iloc[0][['Review Text', 'Review Length', 'Num Tokens']]

By default, if you only type your_dataframe.describe(), the describe method will compute summary statistics on all of the **numeric variables** in your dataframe.

The include parameter enables you to specify what data types to operate on and include in the output descriptive statistics.

Possible arguments to this parameter are:

* 'all' (this will include all variables)
* numpy.number (this will include numeric variables)
* object (this will include string variables)
* 'category' (this will include Pandas category variables)

![picture](https://drive.google.com/uc?id=1Achtqzn3MH8do0roqHNCoEU32v7uPK-1)

<img src=”https://cdn-images-1.medium.com/fit/c/75/75/1*6_fgYnisCa9V21mymySIvA.png” width=”100">

In [ ]:
df.describe(include = [np.number])

Here, the syntax np.number indicates that we want to include numeric variables (i.e., Numpy numerics).

**percentiles** = By default, pandas will include the 25th, 50th, and 75th percentile. However you can tell pandas whichever ones you want. Simply pass a list to percentiles and pandas will do the rest.

**include** = You may want to ‘describe’ all of your columns, or you may just want to do the numeric columns. By default, pandas will only describe your numeric columns. Select ‘all’ to include all columns.

**exclude** = The inverse of include, you can tell pandas which column data types you would like to exclude. Simply pass a list of datatypes you would like to exclude here.

**datetime_is_numeric:** By default pandas will treat your datetimes as objects. Meaning, Pandas will not calculate things like ‘average time/date’. However, if you select datetime_is_numeric=True then pandas will apply the min, max, and percentiles to your datetimes.

In [ ]:
# Let's compute the summary statistics for the string variables.
df.describe(include = [object])

We called the describe() method, and inside the parenthesis, we used the syntax include = [object]. Here, object refers to string variables, so the Pandas describe method computes summary stats for the string columns.

Notice that the statistics that are computed are actually different than the stats for the numeric variables.

For the numeric variables, describe() computes things like the minimum, maximum, mean, percentiles, etc.

But for these string variables, describe() has computed the count, the number of unique values, the most frequent value, and the frequency of the most frequent value.

### Automated EDA
using Pandas Profiling library

In [ ]:
# Let's boost data profiling to the next level and be lazy
profile = prof.ProfileReport(df)

In [ ]:
profile.to_file(output_file= data_folder /'clothing_data_profile.html')

## Exploratory Text Analysis (ETA)

Some of the text features' analysis were recently implemented in pandas profiling. However, a lot of them are still missing. Let's take a look at how to build them ourselves.

In [ ]:
# Let's start with some visuals
!pip install WordCloud

### Wordclouds
Word Cloud is a data visualization technique used for representing text data in which the size of each word indicates its frequency or importance. Significant textual data points can be highlighted using a word cloud. Word clouds are widely used for analyzing data from social network websites.

---

### Learn & play:
> If you want to go fancy: https://www.datacamp.com/community/tutorials/wordcloud-python

In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt 
import seaborn as sns

# Set preferences
sns.set(color_codes=True)

In [ ]:
# Get all review texts
all_reviews = df['Review Text']
all_reviews[-1:]

In [ ]:
# Concatenate strings in the Series/Index with cat() function to have all reviews as a string.
all_reviews.str.cat()

In [ ]:
# Now let's generate a wordcloud for some reviews
wordcloud = WordCloud(width = 500, height = 500, 
                background_color ='white',
                min_font_size = 10).generate(all_reviews.str.cat())
  
# plot the WordCloud image                        
plt.figure(figsize = (6, 6), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
# What departments do we have in the dataset?
df['Department Name'].value_counts()

In [ ]:
# Compare two departments
jackets = df[df['Department Name'] == 'Jackets']['Review Text']
intimate = df[df['Department Name'] == 'Intimate']['Review Text']
print(intimate.str.cat()[:100])

In [ ]:
rows=1
cols=2

row=0 
col=0
titles = ['Jackets', 'Intimate']
w = [jackets.str.cat(),intimate.str.cat(),]
fig, ax = plt.subplots(1, 2, figsize=(12.5,6.5))

for i in range(2):
  ax[col].imshow(WordCloud(width = 500, height = 500, 
                background_color ='white', 
                min_font_size = 10).generate(w[i]))
  ax[col].axis("off")
  ax[col].set_title(titles[i], fontdict={'fontsize': 15, 'fontweight' : 'bold'})
  row=row+1
  if row==rows:
     row=0
     col=col+1

plt.subplots_adjust(left=0.07, right=0.93, wspace=0.1, hspace=0.0,top=0.94,bottom=0.09)
plt.show() 

In [ ]:
# To make this code more scalable, we will automate the wordcloud generation for all categories next.
g = df.groupby(['Department Name'])['Review Text']
for i, k in g:
  i
  print(k.str.cat()[:100])

What can you notice about preprocessing in the above word clouds?

In [ ]:
# Word clouds for all departments
np.random.seed(1)

figure, axes = plt.subplots(2, 3, figsize=(18, 14))
cluster_groups = list(i.str.cat() for k, i in df.groupby(['Department Name'])['Review Text'])
cluster_titles = list(k for k, i in df.groupby(['Department Name'])['Review Text'])
i = 0
for r in range(2):
    for c in range(3):
        df_cluster = cluster_groups[i]
        wordcloud_image = WordCloud(width = 500, height = 500, 
                background_color ='white', 
                min_font_size = 10).generate(df_cluster) 
        ax = axes[r][c]
        ax.imshow(wordcloud_image,
                  interpolation="bilinear")
        ax.set_title(cluster_titles[i], fontsize=20)
        ax.set_xticks([])
        ax.set_yticks([])
        i = i + 1

plt.subplots_adjust(left=0.07, right=0.93, wspace=0.08, hspace=0.0,top=0.9,bottom=0.09)
#plt.tight_layout()
plt.show()

Now that we have some insights about the vocabulary of the reviews, let's dig deeper. We will inspect some of our features.

### Univariate analyses
Single-variable or univariate visualization is the simplest type of visualization which consists of observations on only a single characteristic or attribute. Univariate visualization includes histogram, bar plots and line charts.

#### Polarity
Inspect polarity distribution.

In [ ]:
# Select some reviews with high sentiment polarity
high_pol = df.loc[df['Polarity'] == 1, ['Review Text']].sample(5).values
for r in high_pol:
  print(r[0] + '\n')

In [ ]:
# Select some reviews with low sentiment polarity
low_pol = df.loc[df['Polarity'] < -0.5, ['Review Text']].sample(5).values
for r in low_pol:
    print(r[0]+ '\n')

In [ ]:
df['Polarity'].min()

In [ ]:
df.loc[df['Polarity'] < -0.8]

In [ ]:
# Plot the distribution of polarity sentiment score
# change plotting colors per client request
plt.style.use('ggplot')
df['Polarity'].plot(kind='hist', bins=50)
plt.xlabel('Polarity')
plt.title('Polarity distribution')

#### Knowledge check 🤔  

Why does hist makes sense here?

Vast majority of the polarity are greater than 0, means most of them are positive.

#### Ratings

In [ ]:
df['Rating'].value_counts().sort_index().plot(kind='bar', color='cadetblue')
#colourmaps= 'summer'
plt.xlabel('Rating')
plt.title('Rating distribution')

The ratings are in align with the polarity, that is, most of the ratings are at 4 or 5 range.

#### Age

In [ ]:
df['Age'].plot(kind='hist', bins=20, color='peru')
#colourmaps= 'summer'
plt.xlabel('Age')
plt.title('Age distribution')

Most reviewers are in their 30s to 40s.

### Multivariate analyses



---


### Learn & play:
Always read the documentation!

- [Pandas Plotting Documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)

- [Matplotlib Documentation](https://matplotlib.org/)

- [Matplotlib sample plots](https://matplotlib.org/tutorials/introductory/sample_plots.html)

In [ ]:
df.hist(figsize=(16,8));

#### Knowledge check  🤔  

Why doesn't it make plots of ALL the columns in the dataframe?
Hint: what is different about the columns it plots vs. the ones it left out?

#### Scatter plot

In [ ]:
#Scatter plots are very good at showing the **interaction between two numeric variables** (especially when they're continuous)!
df.plot(kind='scatter', x='Review Length', y='Polarity', \
        color='plum', figsize=(10,4), s=10, alpha=0.5)

#### Scatter matrix
We can also use a thing called a scatter matrix or a pairplot, which is a grid of scatter plots. This allows you to quickly view the interaction of N x M features. You are generally looking for a trend between variables (a line or curve). Using machine learning, you can fit these curves to provide predictive power.

In [ ]:
pd.plotting.scatter_matrix(
    df.select_dtypes(include='number').iloc[:,-5:-1],
    figsize=(12,12)
);

We can also use a very handy parameter, `c`, which allows us to color the dots in a scatter plot. This is extremely helpful when doing **classification problems**, often you will set the color to the class label.

In [ ]:
df['Recommended IND'].unique()

In [ ]:
df.plot(kind='scatter', x='Review Length', y='Polarity', c='Recommended IND', colormap='summer', figsize=(15,7))
plt.ylabel('Length')
plt.ylabel('Polarity')
#plt.savefig('giveaname.png');

Recommended reviews tend to be lengthier than those of not recommended reviews.

#### Boxplot

In [ ]:
# Which department is the best rated?
y0 = df.loc[df['Department Name'] == 'Tops']['Rating']
y1 = df.loc[df['Department Name'] == 'Dresses']['Rating']
y2 = df.loc[df['Department Name'] == 'Bottoms']['Rating']
y3 = df.loc[df['Department Name'] == 'Intimate']['Rating']
y4 = df.loc[df['Department Name'] == 'Jackets']['Rating']
y5 = df.loc[df['Department Name'] == 'Trend']['Rating']

trace0 = go.Box(
    y=y0,
    name = 'Tops',
    marker = dict(
        color = 'rgb(214, 12, 140)',
    )
)
trace1 = go.Box(
    y=y1,
    name = 'Dresses',
    marker = dict(
        color = 'rgb(0, 128, 128)',
    )
)
trace2 = go.Box(
    y=y2,
    name = 'Bottoms',
    marker = dict(
        color = 'rgb(10, 140, 208)',
    )
)
trace3 = go.Box(
    y=y3,
    name = 'Intimate',
    marker = dict(
        color = 'rgb(12, 102, 14)',
    )
)
trace4 = go.Box(
    y=y4,
    name = 'Jackets',
    marker = dict(
        color = 'rgb(10, 0, 100)',
    )
)
trace5 = go.Box(
    y=y5,
    name = 'Trend',
    marker = dict(
        color = 'rgb(100, 0, 10)',
    )
)
data = [trace0, trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(
    title = "Rating Boxplot of Department Name"
)

fig = go.Figure(data=data,layout=layout)
fig.show(renderer="colab")
iplot(fig, filename = "Rating Boxplot of Department Name")

Except Trend department, all the other departments’ median rating were 5. Overall, the ratings are high and sentiment are positive in this review data set.

#### Jointplot

In [ ]:
trace1 = go.Scatter(
    x=df['Age'], y=df['Polarity'], mode='markers', name='points',
    marker=dict(color='rgb(102,0,0)', size=2, opacity=0.4)
)
trace2 = go.Histogram2dContour(
    x=df['Age'], y=df['Polarity'], name='density', ncontours=20,
    colorscale='Hot', reversescale=True, showscale=False
)
trace3 = go.Histogram(
    x=df['Age'], name='Age density',
    marker=dict(color='rgb(102,0,0)'),
    yaxis='y2'
)
trace4 = go.Histogram(
    y=df['Polarity'], name='Sentiment Polarity density', marker=dict(color='rgb(102,0,0)'),
    xaxis='x2'
)
data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
    showlegend=False,
    autosize=False,
    width=1200,
    height=800,
    xaxis=dict(
        domain=[0, 0.85],
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        domain=[0, 0.85],
        showgrid=False,
        zeroline=False
    ),
    margin=dict(
        t=50
    ),
    hovermode='closest',
    bargap=0,
    xaxis2=dict(
        domain=[0.85, 1],
        showgrid=False,
        zeroline=False
    ),
    yaxis2=dict(
        domain=[0.85, 1],
        showgrid=False,
        zeroline=False
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='2dhistogram-2d-density-plot-subplots')
fig.show(renderer="colab")

There were few people are very positive or very negative. People who give neutral to positive reviews are more likely to be in their 30s. Probably people at these age are likely to be more active.

In [ ]:
df.info()

#### Correlation matrix
Compute pairwise correlation of columns, excluding NA/null values.

Parameters
method {‘pearson’, ‘kendall’, ‘spearman’} or callable
Method of correlation:

pearson : standard correlation coefficient

kendall : Kendall Tau correlation coefficient

spearman : Spearman rank correlation

callable: callable with input two 1d ndarrays
and returning a float. Note that the returned matrix from corr will have 1 along the diagonals and will be symmetric regardless of the callable’s behavior.

min_periodsint, optional
Minimum number of observations required per pair of columns to have a valid result. Currently only available for Pearson and Spearman correlation.

In [ ]:
correlation = df[['Rating','Polarity', 'Review Length', 'Num Tokens']].corr()
mask = np.zeros_like(correlation, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(12,10))
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
sns.heatmap(correlation, cmap='coolwarm', annot=True, annot_kws={"size": 20}, linewidths=10, vmin=-1.5, mask=mask)

### N-gramming

N-grams are used to describe the number of words used as observation points, e.g., unigram means singly-worded, bigram means 2-worded phrase, and trigram means 3-worded phrase. In order to do this, we use scikit-learn’s CountVectorizer function.

#### Unigrams

In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_words(df['Review Text'], 20)

unigrams_w_stops = pd.DataFrame(common_words, columns = ['word' , 'count'])
unigrams_w_stops

In [ ]:
unigrams_w_stops.plot(kind='barh', x='word', color='cadetblue', width=0.5, figsize=(5,6))
plt.xlabel('count')
plt.ylabel('word')
plt.title('Unigrams with stopwords')

In [ ]:
# Unigrams after stopword removal
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['Review Text'], 20)
unigrams = pd.DataFrame(common_words, columns = ['word' , 'count'])
unigrams

In [ ]:
unigrams.plot(kind='barh', x='word', color='cadetblue', width=0.5, figsize=(5,6))
plt.xlabel('count')
plt.ylabel('word')
plt.title('Unigrams w/o stopwords')

#### Bigrams

In [ ]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['Review Text'], 20)
bigrams = pd.DataFrame(common_words, columns = ['word' , 'count'])

In [ ]:
bigrams.plot(kind='barh', x='word', color='cadetblue', width=0.5, figsize=(5,6))
plt.xlabel('count')
plt.ylabel('word')
plt.title('Bigrams w/o stopwords')

#### Trigrams

In [ ]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['Review Text'], 20)
trigrams = pd.DataFrame(common_words, columns = ['word' , 'count'])

In [ ]:
trigrams.plot(kind='barh', x='word', color='cadetblue', width=0.5, figsize=(5,6))
plt.xlabel('count')
plt.ylabel('word')
plt.title('Trigrams w/o stopwords')

We will talk about n-gram more when we learn about vector representations and classification algorithms. For now, just remember that ngrams can carry special meaning. For instance, data & science might add up closely to data science. However, honeymoon has nothing to do with honey or moon.